<a href="https://colab.research.google.com/github/Xubwei/NLP_Bert/blob/main/test_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import DistilBertTokenizer
from transformers import DistilBertModel
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification

import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from transformers import DistilBertForSequenceClassification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 建立模型
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to(device)

# 載入模型權重
state_dict = torch.load('model_best_v5.pt')
model.load_state_dict(state_dict)

model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
df = pd.read_csv("BENQ標記資料_Final.csv")
df['result'] = df['result'].astype('category')
df['result'] = df['result'].map({0.0: 1, 1.0: 0})
df = df.dropna()
df.head()

,content,result
0,I was so happy tonight. My wife said I had a n...,1
1,usb port stopped working\n from one day to an...,1
2,Over priced\n In darkness it's a great pictur...,1
3,Outstanding like being in a theater!\n I boug...,1
4,"The best projector ever, read on??\n Solid co...",1


In [ ]:
df.shape

(9889, 2)

### 隨機抽樣1000筆進行測試

In [ ]:
test_df = df.sample(n=1000, random_state=42)
test_df.head()

,content,result
4468,"Weekly Roundup: Q&A, New Releases, and Promos!...",0
4823,"Projection Mobile Phone, All-In-One Smart HD P...",0
7735,50% Off TG90簞 4500mah Small Portable Power Ban...,0
733,"Hello friends, I am trying to connect my surro...",1
7659,"50% Off Smart Projector Android TV 9.0, Artlii...",0


In [ ]:
test_df.shape

(1000, 2)

### 1.嘗試使用logtis預測


In [ ]:
# Initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

# Iterate over the content in the CSV file and make predictions
predictions = []
for content in test_df['content']: # 這裡暫時放 comparison_df_prob
    # Encode the content
    encoded_input = tokenizer(content, truncation=True, padding=True, return_tensors='pt')
    encoded_input = encoded_input.to('cuda' if torch.cuda.is_available() else 'cpu')

    # Make a prediction
    with torch.no_grad():
        outputs = model(**encoded_input)
        logits = outputs.logits

        # Get the predicted label
        _, predicted_label = torch.max(logits, dim=1)
        predictions.append(predicted_label.item())

# Add the predictions to the DataFrame
test_df['predicted_logits_v1'] = predictions
test_df.head()

,content,result,predicted_logits_v1
4468,"Weekly Roundup: Q&A, New Releases, and Promos!...",0,1
4823,"Projection Mobile Phone, All-In-One Smart HD P...",0,0
7735,50% Off TG90簞 4500mah Small Portable Power Ban...,0,0
733,"Hello friends, I am trying to connect my surro...",1,1
7659,"50% Off Smart Projector Android TV 9.0, Artlii...",0,0


### 2.嘗試使用predict model 預測

In [ ]:
import time

# Prepare the data
PRE_TRAINED_MODEL_NAME = 'distilbert-base-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)
texts = test_df['content'].tolist()
encodings = tokenizer(texts, truncation=True, padding=True)
input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])

# Split the data into batches
batch_size = 50  # Adjust as needed
input_batches = torch.split(input_ids, batch_size)
attention_mask_batches = torch.split(attention_mask, batch_size)

# Make the predictions
predictions = []
for i, (input_batch, attention_mask_batch) in enumerate(zip(input_batches, attention_mask_batches)):
    start_time = time.time()  # Get the start time
    with torch.no_grad():
        outputs = model(input_batch.to(device), attention_mask=attention_mask_batch.to(device))
        _, batch_predictions = torch.max(outputs['logits'], 1)
        predictions.extend(batch_predictions.cpu().numpy())
    end_time = time.time()  # Get the end time
    print(f"Batch {i+1} of {len(input_batches)}: Time taken: {end_time - start_time:.2f} seconds")  # Print the time taken

# Print the predictions

test_df['prediction_model'] = predictions

Batch 1 of 20: Time taken: 1.88 seconds
Batch 2 of 20: Time taken: 1.82 seconds
Batch 3 of 20: Time taken: 1.83 seconds
Batch 4 of 20: Time taken: 1.82 seconds
Batch 5 of 20: Time taken: 1.84 seconds
Batch 6 of 20: Time taken: 1.83 seconds
Batch 7 of 20: Time taken: 1.84 seconds
Batch 8 of 20: Time taken: 1.84 seconds
Batch 9 of 20: Time taken: 1.84 seconds
Batch 10 of 20: Time taken: 1.83 seconds
Batch 11 of 20: Time taken: 1.84 seconds
Batch 12 of 20: Time taken: 1.84 seconds
Batch 13 of 20: Time taken: 1.82 seconds
Batch 14 of 20: Time taken: 1.84 seconds
Batch 15 of 20: Time taken: 1.84 seconds
Batch 16 of 20: Time taken: 1.84 seconds
Batch 17 of 20: Time taken: 1.84 seconds
Batch 18 of 20: Time taken: 1.84 seconds
Batch 19 of 20: Time taken: 1.85 seconds
Batch 20 of 20: Time taken: 1.84 seconds


In [ ]:
test_df.head()

,content,result,predicted_logits_v1,prediction_model
4468,"Weekly Roundup: Q&A, New Releases, and Promos!...",0,1,1
4823,"Projection Mobile Phone, All-In-One Smart HD P...",0,0,0
7735,50% Off TG90簞 4500mah Small Portable Power Ban...,0,0,0
733,"Hello friends, I am trying to connect my surro...",1,1,1
7659,"50% Off Smart Projector Android TV 9.0, Artlii...",0,0,0


### 3. 使用prob看

In [ ]:
# 載入tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# 定義一個函數，該函數接受一段文本，並返回該文本為非廣告的機率
def get_prob_non_ad(text):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}

    outputs = model(**inputs)
    scores = outputs.logits

    # 將原始的分數通過softmax函數轉換為機率。
    probs = torch.nn.functional.softmax(scores, dim=-1)

    # 將預測結果的機率從GPU取出來並轉換為numpy array，並取第一個元素的第二個值，這裡是代表「非廣告文」的機率。
    prob_non_ad = probs.detach().to('cpu').numpy()[0,1]

    return prob_non_ad

# 為DataFrame的每一行增加一個新的欄位
test_df['prob_non_ad'] = test_df['content'].apply(get_prob_non_ad)

In [ ]:
test_df.head()

,content,result,predicted_logits_v1,prediction_model,prob_non_ad
4468,"Weekly Roundup: Q&A, New Releases, and Promos!...",0,1,1,0.714316
4823,"Projection Mobile Phone, All-In-One Smart HD P...",0,0,0,0.534269
7735,50% Off TG90簞 4500mah Small Portable Power Ban...,0,0,0,0.527293
733,"Hello friends, I am trying to connect my surro...",1,1,1,0.463084
7659,"50% Off Smart Projector Android TV 9.0, Artlii...",0,0,0,0.432691


### 查看成果

In [ ]:
test_df = test_df.rename(columns={'result': 'True_Label'})
test_df['True_Label'] = test_df['True_Label'].astype(int)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 4468 to 9236
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   content              1000 non-null   object 
 1   True_Label           1000 non-null   int64  
 2   predicted_logits_v1  1000 non-null   int64  
 3   prediction_model     1000 non-null   int64  
 4   prob_non_ad          1000 non-null   float32
dtypes: float32(1), int64(3), object(1)
memory usage: 43.0+ KB


In [ ]:
test_df.head()

,content,True_Label,predicted_logits_v1,prediction_model,prob_non_ad
4468,"Weekly Roundup: Q&A, New Releases, and Promos!...",0,1,1,0.714316
4823,"Projection Mobile Phone, All-In-One Smart HD P...",0,0,0,0.534269
7735,50% Off TG90簞 4500mah Small Portable Power Ban...,0,0,0,0.527293
733,"Hello friends, I am trying to connect my surro...",1,1,1,0.463084
7659,"50% Off Smart Projector Android TV 9.0, Artlii...",0,0,0,0.432691


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np

# Create a dictionary to store the results
results = {}

# Calculate metrics for each prediction method
for method in ['prob_non_ad', 'predicted_logits_v1', 'prediction_model']:
    if method == 'prob_non_ad':
        # Use 0.5 as the threshold
        predictions = np.where(test_df[method] >= 0.5, 1, 0)
    else:
        predictions = test_df[method]

    accuracy = accuracy_score(test_df['True_Label'], predictions)
    precision = precision_score(test_df['True_Label'], predictions)
    recall = recall_score(test_df['True_Label'], predictions)
    f1 = f1_score(test_df['True_Label'], predictions)

    results[method] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Convert the results dictionary to a dataframe
results_df = pd.DataFrame(results).T

# Display the results dataframe
results_df

,accuracy,precision,recall,f1
prob_non_ad,0.374,0.313830,0.682081,0.429872
predicted_logits_v1,0.924,0.870879,0.916185,0.892958
prediction_model,0.924,0.870879,0.916185,0.892958


### 詳細查看 predicted_logits_v1

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
new_predictions = (test_df['predicted_logits_v1']).astype(int)
# Compute new accuracy
new_accuracy = accuracy_score(test_df['True_Label'], new_predictions)
# Compute new confusion matrix
new_confusion_mat = confusion_matrix(test_df['True_Label'], new_predictions)
# Compute new classification report
new_class_report = classification_report(test_df['True_Label'], new_predictions)
print(new_accuracy)
print("=================")
print(new_confusion_mat)
print("=================")
print(new_class_report)

0.924
[[607  47]
 [ 29 317]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       654
           1       0.87      0.92      0.89       346

    accuracy                           0.92      1000
   macro avg       0.91      0.92      0.92      1000
weighted avg       0.93      0.92      0.92      1000

